In [4]:
import pandas as pd
import os
import glob
import numpy as np

print("==================================================")
print("   ODDS DATA UNIFIER (MULTI-LEAGUE MERGE)         ")
print("==================================================")

# 1. Define the files you have
# Put your filenames here
files = [
    "odds_data/D1 (1).csv", "odds_data/D1.csv",
    "odds_data/E0 (1).csv", "odds_data/E0.csv",
    "odds_data/F1 (1).csv", "odds_data/F1.csv",
    "odds_data/I1 (1).csv", "odds_data/I1.csv",
    "odds_data/SP1 (1).csv", "odds_data/SP1.csv"
]

# 2. Define the Columns we actually need
# We prioritize 'Avg' (Market Average) but fallback to 'B365' (Bet365) if missing
target_cols = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']
odds_priority = [
    ('AvgH', 'B365H'), ('AvgD', 'B365D'), ('AvgA', 'B365A'), # 1X2
    ('Avg>2.5', 'B365>2.5'), ('Avg<2.5', 'B365<2.5')         # Over/Under
]

master_list = []

for f in files:
    if not os.path.exists(f):
        print(f"⚠️ Warning: File '{f}' not found. Skipping.")
        continue
        
    print(f"Processing {f}...")
    try:
        # Load with robust parsing
        df = pd.read_csv(f, encoding='unicode_escape', on_bad_lines='skip')
        
        # Standardize Columns
        # Create a new mini-dataframe with standardized names
        clean_df = df[target_cols].copy()
        
        # Smart Column Selection (Avg vs B365)
        for avg_col, backup_col in odds_priority:
            if avg_col in df.columns:
                clean_df[avg_col] = df[avg_col] # Use Average if available
            elif backup_col in df.columns:
                clean_df[avg_col] = df[backup_col] # Use Bet365 as fallback
            else:
                clean_df[avg_col] = np.nan # Missing
        
        # Ensure date format is consistent
        # Football-Data.co.uk usually uses dd/mm/yyyy or dd/mm/yy
        clean_df['Date'] = pd.to_datetime(clean_df['Date'], dayfirst=True, errors='coerce')
        
        # Drop rows with no date or teams
        clean_df = clean_df.dropna(subset=['Date', 'HomeTeam', 'AwayTeam'])
        
        master_list.append(clean_df)
        
    except Exception as e:
        print(f"❌ Error reading {f}: {e}")

# 3. Concatenate
if master_list:
    big_odds = pd.concat(master_list, ignore_index=True)
    
    # Save as the master file expected by your merge script
    big_odds.to_csv("odds_data.csv", index=False)
    print("\n✅ SUCCESS: Created 'odds_data.csv'")
    print(f"   Total Matches: {len(big_odds)}")
    print("   Columns: Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, AvgH, AvgD, AvgA, Avg>2.5, Avg<2.5")
else:
    print("\n❌ No data found.")

   ODDS DATA UNIFIER (MULTI-LEAGUE MERGE)         
Processing odds_data/D1 (1).csv...
Processing odds_data/D1.csv...
Processing odds_data/E0 (1).csv...
Processing odds_data/E0.csv...
Processing odds_data/F1 (1).csv...
Processing odds_data/F1.csv...
Processing odds_data/I1 (1).csv...
Processing odds_data/I1.csv...
Processing odds_data/SP1 (1).csv...
Processing odds_data/SP1.csv...

✅ SUCCESS: Created 'odds_data.csv'
   Total Matches: 2597
   Columns: Date, HomeTeam, AwayTeam, FTHG, FTAG, FTR, AvgH, AvgD, AvgA, Avg>2.5, Avg<2.5


In [ ]:
import pandas as pd
import numpy as np
import difflib
import re
import warnings

warnings.filterwarnings('ignore')

print("==================================================")
print("   ODDS INTEGRATION ENGINE (FUZZY MATCHING)       ")
print("==================================================")

# 1. LOAD DATA
try:
    df_stats = pd.read_csv("match_data.csv")
    # Robust load for odds (skip bad lines if files are messy)
    df_odds = pd.read_csv("odds_data.csv", encoding='unicode_escape', on_bad_lines='skip')
    print(f"✅ Loaded Stats: {len(df_stats)} | Odds: {len(df_odds)}")
except FileNotFoundError:
    print("❌ Error: Missing csv files.")
    exit()

# 2. DATE PARSING (Crucial for merging)
# FBref Date
def parse_fbref_date(url):
    try:
        match = re.search(r'([A-Za-z]+-\d{1,2}-\d{4})', str(url))
        if match: return pd.to_datetime(match.group(1), format='%B-%d-%Y')
    except: pass
    return pd.NaT

df_stats['date_obj'] = df_stats['match_url'].apply(parse_fbref_date)

# Odds Date (Handle multiple formats usually found in football-data.co.uk)
df_odds['Date'] = pd.to_datetime(df_odds['Date'], dayfirst=True, errors='coerce')

# 3. TEAM NAME MAPPING
# We map Odds Names -> FBref Names
fbref_teams = sorted(df_stats['home_team_name'].unique())
odds_teams = sorted(df_odds['HomeTeam'].dropna().unique())

team_map = {}

# Manual overrides for known mismatches (Premier League examples)
manual_map = {
    "Man United": "Manchester United",
    "Man City": "Manchester City",
    "Nott'm Forest": "Nottingham Forest",
    "Spurs": "Tottenham Hotspur",
    "Wolves": "Wolverhampton Wanderers",
    "Brighton": "Brighton & Hove Albion",
    "West Ham": "West Ham United",
    "Leeds": "Leeds United", 
    "Leicester": "Leicester City",
    "Newcastle": "Newcastle United"
}

print("Mapping teams...")
for o_team in odds_teams:
    if o_team in manual_map:
        team_map[o_team] = manual_map[o_team]
    else:
        # Fuzzy Match
        matches = difflib.get_close_matches(o_team, fbref_teams, n=1, cutoff=0.6)
        if matches:
            team_map[o_team] = matches[0]

df_odds['mapped_home'] = df_odds['HomeTeam'].map(team_map)

# 4. MERGE
# We merge on 'Date' and 'Home Team'
print("Merging data...")
merged = pd.merge(
    df_stats, 
    df_odds[['Date', 'mapped_home', 'AvgH', 'AvgD', 'AvgA', 'B365H', 'B365D', 'B365A']], 
    left_on=['date_obj', 'home_team_name'], 
    right_on=['Date', 'mapped_home'], 
    how='left' # Keep all stats, fill missing odds with NaN
)

# 5. FEATURE ENGINEERING WITH ODDS
# Prioritize Average Odds, fallback to Bet365
merged['odds_home'] = merged['AvgH'].fillna(merged['B365H'])
merged['odds_draw'] = merged['AvgD'].fillna(merged['B365D'])
merged['odds_away'] = merged['AvgA'].fillna(merged['B365A'])

# Convert to IMPLIED PROBABILITY (The Feature)
# 1 / Odds = Probability
merged['market_prob_home'] = 1 / merged['odds_home']
merged['market_prob_draw'] = 1 / merged['odds_draw']
merged['market_prob_away'] = 1 / merged['odds_away']

# Fill Missing Odds (for matches not in odds database) with Neutral 33%
# Add a flag so the model knows these are fake
merged['has_odds'] = merged['odds_home'].notna().astype(int)
merged[['market_prob_home', 'market_prob_draw', 'market_prob_away']] = merged[['market_prob_home', 'market_prob_draw', 'market_prob_away']].fillna(0.33)

# Drop temp columns
merged = merged.drop(columns=['Date', 'mapped_home', 'date_obj', 'AvgH', 'B365H'])

print(f"✅ Final Dataset: {len(merged)} matches.")
print(f"   Matches with valid odds found: {merged['has_odds'].sum()}")

merged.to_csv("match_data_with_odds.csv", index=False)
print("💾 Saved to 'match_data_with_odds.csv'")

   ODDS INTEGRATION: SMART FUZZY MERGE            
Stats: 2964 | Odds: 2597
Running Smart Merge (Time tolerance +/- 1 day)...
✅ Matched 1947 games with odds.
ℹ️ 1017 games have NO odds (Training will handle this).
💾 Saved to 'match_data_combined.csv'
